### This script contains the below:
1. Import libraries
2. Importing bike trips data
3. Data Exploration
4. Get weather data using NOAA's API
5. Merge bike trip and weather data

# 01 Importing libraries

In [93]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime

# 02 Importing data & merge

In [94]:
# The code below automates the process of importing and combining all monthly CitiBike CSV files for 2022.
# 1. First, it walks through the folder "Original Data" and creates a list (filepaths) containing the full paths 
#    of all CSV files in the folder and its subfolders.
# 2. Then, it uses a generator expression inside pd.concat() to read each CSV file into a temporary DataFrame.
# 3. All these monthly DataFrames are merged (stacked vertically) into one large DataFrame called 'df'.
# 4. The parameter 'ignore_index=True' resets the row index after concatenation, creating a clean continuous index.
# 5. 'low_memory=False' ensures that pandas reads large CSVs more reliably when columns have mixed data types.

In [95]:
# Create a list with all CSV files in the folder using a list comprehension
# The comprehension iterates through all directories and subdirectories with os.walk(),
# checks if each file ends with '.csv', and joins the folder path with the file name.

folderpath = r'../02 Data/Original Data'
filepaths = [
    os.path.join(root, name)
    for root, dirs, files in os.walk(folderpath)
    for name in files
    if name.endswith('.csv')
]

In [96]:
print(len(filepaths))

36


In [97]:
filepaths

['../02 Data/Original Data/202208-citibike-tripdata_3.csv',
 '../02 Data/Original Data/202207-citibike-tripdata_2.csv',
 '../02 Data/Original Data/202207-citibike-tripdata_3.csv',
 '../02 Data/Original Data/202208-citibike-tripdata_2.csv',
 '../02 Data/Original Data/202207-citibike-tripdata_1.csv',
 '../02 Data/Original Data/202208-citibike-tripdata_1.csv',
 '../02 Data/Original Data/202210-citibike-tripdata_1.csv',
 '../02 Data/Original Data/202207-citibike-tripdata_4.csv',
 '../02 Data/Original Data/202208-citibike-tripdata_4.csv',
 '../02 Data/Original Data/202203-citibike-tripdata_2.csv',
 '../02 Data/Original Data/202210-citibike-tripdata_2.csv',
 '../02 Data/Original Data/202203-citibike-tripdata_1.csv',
 '../02 Data/Original Data/202210-citibike-tripdata_3.csv',
 '../02 Data/Original Data/202204-citibike-tripdata_1.csv',
 '../02 Data/Original Data/202204-citibike-tripdata_2.csv',
 '../02 Data/Original Data/202204-citibike-tripdata_3.csv',
 '../02 Data/Original Data/202211-citibi

In [98]:
# merging the data
df = pd.concat((pd.read_csv(f, low_memory=False) for f in filepaths), ignore_index=True)

In [99]:
# Merging the data:
# This line uses a generator expression inside pd.concat() to combine all CSV files into one DataFrame.
# The generator (pd.read_csv(f) for f in filepaths) loads each file one by one instead of all at once,
# which makes the process more memory-efficient for large datasets.

In [100]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual
1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual
2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member
3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.10,40.744219,-73.971212,40.764397,-73.973715,member
4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member


In [101]:
# Convert the columns to datetime (if not already)
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

# Get the earliest and latest dates for each column
earliest_start = df['started_at'].min()
latest_start = df['started_at'].max()

earliest_end = df['ended_at'].min()
latest_end = df['ended_at'].max()

# Print the results
print("Earliest start date:", earliest_start)
print("Latest start date:", latest_start)
print("Earliest end date:", earliest_end)
print("Latest end date:", latest_end)

Earliest start date: 2021-01-30 17:30:45.544000
Latest start date: 2022-12-31 23:58:19.206000
Earliest end date: 2022-01-01 00:00:09.459000
Latest end date: 2022-12-31 23:59:55.708000


In [102]:
df2022 = df[(df['started_at'] >= '2022-01-01') & (df['started_at'] <= '2022-12-31')].copy()

In [103]:
df2022.shape

(29806700, 13)

In [104]:
df2022.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29806700 entries, 0 to 29838805
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 3.1+ GB


In [105]:
cat_cols = [
    'rideable_type',
    'start_station_name',
    'start_station_id',
    'end_station_name',
    'end_station_id',
    'member_casual'
]

for col in cat_cols:
    df2022[col] = df2022[col].astype('category')

In [106]:
float_cols = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
df2022[float_cols] = df2022[float_cols].apply(pd.to_numeric, downcast='float')

In [107]:
df2022.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29806700 entries, 0 to 29838805
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       category      
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  category      
 5   start_station_id    category      
 6   end_station_name    category      
 7   end_station_id      category      
 8   start_lat           float32       
 9   start_lng           float32       
 10  end_lat             float32       
 11  end_lng             float32       
 12  member_casual       category      
dtypes: category(6), datetime64[ns](2), float32(4), object(1)
memory usage: 1.6+ GB


In [108]:
summary = pd.DataFrame({
    'dtype': df2022.dtypes,
    'non_nulls': df2022.notnull().sum(),
    'nulls': df2022.isnull().sum(),
    'unique': df2022.nunique()
})

summary

,dtype,non_nulls,nulls,unique
ride_id,object,29806700,0,29806700
rideable_type,category,29806700,0,2
started_at,datetime64[ns],29806700,0,29801411
ended_at,datetime64[ns],29806700,0,29771187
start_station_name,category,29806653,47,1761
start_station_id,category,29806653,47,3468
end_station_name,category,29736910,69790,1840
end_station_id,category,29736910,69790,1905
start_lat,float32,29806700,0,57135
start_lng,float32,29806700,0,17996


In [109]:
# Save the rides for the 2022 year to a pickle file
df2022.to_pickle('df2022.pkl')

# 03 Data exploration

In [110]:
# Counts of bike types
df2022['rideable_type'].value_counts()

classic_bike     18092636
electric_bike    11714064
Name: rideable_type, dtype: int64

In [111]:
# === RESEARCH QUESTIONS ===

# Q1: What are the most popular stations in the city?
print("\n=== Q1: Top 10 Most Popular Start Stations ===")
popular_stations = df2022['start_station_name'].value_counts().head(10)
display(popular_stations)

print("\n=== Q1b: Top 10 Most Popular End Stations ===")
popular_end_stations = df2022['end_station_name'].value_counts().head(10)
display(popular_end_stations)


=== Q1: Top 10 Most Popular Start Stations ===


W 21 St & 6 Ave            128885
West St & Chambers St      123239
Broadway & W 58 St         114056
6 Ave & W 33 St            106352
1 Ave & E 68 St            104782
Broadway & E 14 St          98759
Broadway & W 25 St          98344
University Pl & E 14 St     97013
Broadway & E 21 St          95606
W 31 St & 7 Ave             94111
Name: start_station_name, dtype: int64


=== Q1b: Top 10 Most Popular End Stations ===


W 21 St & 6 Ave            130014
West St & Chambers St      124286
Broadway & W 58 St         110063
1 Ave & E 68 St            105040
6 Ave & W 33 St            104827
University Pl & E 14 St     99072
Broadway & W 25 St          98670
Broadway & E 14 St          98172
Broadway & E 21 St          95423
W 31 St & 7 Ave             94023
Name: end_station_name, dtype: int64

Q1. Most Popular Stations

The analysis reveals a strong overlap between the most frequently used start and end stations in New York’s CitiBike network.
W 21 St & 6 Ave stands out as the single most popular location for both trip departures and arrivals.

The data highlights the central Manhattan corridor as the hub of CitiBike usage, reflecting its role as the city’s primary commuting and tourist zone.

In [112]:
# Q2: Which are the months with the most trips taken?
# Create a month column
df2022['month'] = df2022['started_at'].dt.month_name()

print("\n=== Q2: Trips by Month ===")
trips_by_month = df2022['month'].value_counts()
display(trips_by_month)


=== Q2: Trips by Month ===


August       3576182
September    3411909
July         3397722
June         3344145
October      2935959
May          2865301
November     2386350
April        2261339
March        1846035
December     1560344
February     1197359
January      1024055
Name: month, dtype: int64

The data suggests that summer and early autumn represent the core operating season for the CitiBike network, while winter months show reduced demand.

In [113]:
# Q3: What are the most popular trips between stations?
print("\n=== Q3: Top 10 Most Common Start→End Station Pairs ===")
popular_routes = (
    df2022.groupby(['start_station_name', 'end_station_name'])
    .size()
    .sort_values(ascending=False)
    .head(10)
)
display(popular_routes)


=== Q3: Top 10 Most Common Start→End Station Pairs ===


start_station_name                 end_station_name                 
Central Park S & 6 Ave             Central Park S & 6 Ave               12041
7 Ave & Central Park South         7 Ave & Central Park South            8541
Roosevelt Island Tramway           Roosevelt Island Tramway              8211
Soissons Landing                   Soissons Landing                      7275
Grand Army Plaza & Central Park S  Grand Army Plaza & Central Park S     7259
W 21 St & 6 Ave                    9 Ave & W 22 St                       6336
5 Ave & E 72 St                    5 Ave & E 72 St                       6031
1 Ave & E 62 St                    1 Ave & E 68 St                       5818
Yankee Ferry Terminal              Yankee Ferry Terminal                 5759
Broadway & W 58 St                 Broadway & W 58 St                    5475
dtype: int64

The data highlights Central Park and Roosevelt Island as dominant recreational cycling areas

In [114]:
# Q4: Are the existing stations evenly distributed?
# Check the geographic spread using latitude/longitude ranges
print("\n=== Q4: Station Geographic Distribution ===")
print("Start stations latitude range:", df2022['start_lat'].min(), "-", df2022['start_lat'].max())
print("Start stations longitude range:", df2022['start_lng'].min(), "-", df2022['start_lng'].max())

print("Number of unique start stations:", df2022['start_station_id'].nunique())
print("Number of unique end stations:", df2022['end_station_id'].nunique())


=== Q4: Station Geographic Distribution ===
Start stations latitude range: 40.62737 - 40.88398
Start stations longitude range: -74.028015 - -73.878586
Number of unique start stations: 3468
Number of unique end stations: 1905


Summary: Station Geographic Distribution (2022)

CitiBike stations in 2022 were widely distributed across New York City, covering a latitude range from 40.63° to 40.88° and a longitude range from –74.03° to –73.88°.

This area spans from southern Brooklyn and Jersey City up to northern Manhattan and the Bronx, reflecting the network’s broad urban coverage.

There were 3,468 unique start stations and 1,904 unique end stations, suggesting some newly installed or temporarily inactive stations may not yet appear in both categories.

Overall, the data shows that CitiBike’s infrastructure is densely concentrated in central areas but extends across multiple boroughs, supporting both commuter and recreational use across the city.

# 04 Obtain weather data from New York LaGuardia’s weather station for 2022 and export it to a CSV file.

In [115]:
# Define your NOAA token

Token = 'HLPKfjkSRPFNvFLMQBdRxcFpoojZoEwc'

In [116]:
import requests

# --- WINTER (January–February + December 2022) ---
url_winter_1 = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"   # mean daily temperature
    "&datatypeid=TMAX"   # max daily temperature
    "&datatypeid=TMIN"   # min daily temperature
    "&datatypeid=PRCP"   # daily precipitation
    "&datatypeid=AWND"   # average wind speed
    "&startdate=2022-01-01"
    "&enddate=2022-02-28"
    "&limit=1000"
)
r_winter_1 = requests.get(url_winter_1, headers={"token": Token})
print("Winter part 1 (Jan–Feb):", r_winter_1.status_code)

url_winter_2 = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-12-01"
    "&enddate=2022-12-31"
    "&limit=1000"
)
r_winter_2 = requests.get(url_winter_2, headers={"token": Token})
print("Winter part 2 (Dec):", r_winter_2.status_code)


# --- SPRING (March–May 2022) ---
url_spring = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-03-01"
    "&enddate=2022-05-31"
    "&limit=1000"
)
r_spring = requests.get(url_spring, headers={"token": Token})
print("Spring (Mar–May):", r_spring.status_code)


# --- SUMMER (June–August 2022) ---
url_summer = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-06-01"
    "&enddate=2022-08-31"
    "&limit=1000"
)
r_summer = requests.get(url_summer, headers={"token": Token})
print("Summer (Jun–Aug):", r_summer.status_code)


# --- FALL (September–November 2022) ---
url_fall = (
    "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    "?datasetid=GHCND"
    "&stationid=GHCND:USW00014732"
    "&datatypeid=TAVG"
    "&datatypeid=TMAX"
    "&datatypeid=TMIN"
    "&datatypeid=PRCP"
    "&datatypeid=AWND"
    "&startdate=2022-09-01"
    "&enddate=2022-11-30"
    "&limit=1000"
)
r_fall = requests.get(url_fall, headers={"token": Token})
print("Fall (Sep–Nov):", r_fall.status_code)

Winter part 1 (Jan–Feb): 200
Winter part 2 (Dec): 200
Spring (Mar–May): 200
Summer (Jun–Aug): 200
Fall (Sep–Nov): 200


In [117]:
# Combine all JSON results into one list
all_results = (
    r_winter_1.json()['results'] +
    r_winter_2.json()['results'] +
    r_spring.json()['results'] +
    r_summer.json()['results'] +
    r_fall.json()['results']
)

# Convert to a DataFrame
weather_raw = pd.DataFrame(all_results)

# Check structure
print("✅ Total records collected:", len(weather_raw))
print(weather_raw.head())

✅ Total records collected: 1825
                  date datatype            station attributes  value
0  2022-01-01T00:00:00     AWND  GHCND:USW00014732       ,,W,     28
1  2022-01-01T00:00:00     PRCP  GHCND:USW00014732   ,,W,2400    193
2  2022-01-01T00:00:00     TAVG  GHCND:USW00014732      H,,S,    116
3  2022-01-01T00:00:00     TMAX  GHCND:USW00014732   ,,W,2400    139
4  2022-01-01T00:00:00     TMIN  GHCND:USW00014732   ,,W,2400    100


In [118]:
# Pivot the raw table: each datatype becomes a column
weather_df = weather_raw.pivot(index='date', columns='datatype', values='value').reset_index()

# Rename columns for clarity
weather_df.rename(columns={
    'TAVG': 'avgTemp_tenthsC',
    'TMAX': 'temp_max_tenthsC',
    'TMIN': 'temp_min_tenthsC',
    'PRCP': 'total_precip_tenthsMM',
    'AWND': 'wind_speed_tenthsMS'
}, inplace=True)

# Convert NOAA "tenths" units to normal numeric units
weather_df['avgTemp'] = weather_df['avgTemp_tenthsC'] / 10     # °C
weather_df['temp_max'] = weather_df['temp_max_tenthsC'] / 10   # °C
weather_df['temp_min'] = weather_df['temp_min_tenthsC'] / 10   # °C
weather_df['total_precipitation'] = weather_df['total_precip_tenthsMM'] / 10  # mm
weather_df['wind_speed'] = weather_df['wind_speed_tenthsMS'] / 10             # m/s

# Keep only the useful columns
weather_df = weather_df[['date', 'avgTemp', 'temp_max', 'temp_min', 'total_precipitation', 'wind_speed']]

# Convert date to datetime and sort
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_df = weather_df.sort_values('date').reset_index(drop=True)

# Check result
print("✅ Weather DataFrame created:", weather_df.shape)
print(weather_df.head())

✅ Weather DataFrame created: (365, 6)
datatype       date  avgTemp  temp_max  temp_min  total_precipitation  \
0        2022-01-01     11.6      13.9      10.0                 19.3   
1        2022-01-02     11.4      15.6       3.9                  1.0   
2        2022-01-03      1.4       3.9      -4.3                  0.0   
3        2022-01-04     -2.7       2.2      -6.0                  0.0   
4        2022-01-05      3.2       8.9       0.0                  6.1   

datatype  wind_speed  
0                2.8  
1                4.3  
2                6.4  
3                3.9  
4                3.4  


In [119]:
weather_df.shape

(365, 6)

In [120]:
weather_df.head()

datatype,date,avgTemp,temp_max,temp_min,total_precipitation,wind_speed
0,2022-01-01,11.6,13.9,10.0,19.3,2.8
1,2022-01-02,11.4,15.6,3.9,1.0,4.3
2,2022-01-03,1.4,3.9,-4.3,0.0,6.4
3,2022-01-04,-2.7,2.2,-6.0,0.0,3.9
4,2022-01-05,3.2,8.9,0.0,6.1,3.4


In [121]:
weather_df.columns.name = None

In [122]:
weather_df.head()

,date,avgTemp,temp_max,temp_min,total_precipitation,wind_speed
0,2022-01-01,11.6,13.9,10.0,19.3,2.8
1,2022-01-02,11.4,15.6,3.9,1.0,4.3
2,2022-01-03,1.4,3.9,-4.3,0.0,6.4
3,2022-01-04,-2.7,2.2,-6.0,0.0,3.9
4,2022-01-05,3.2,8.9,0.0,6.1,3.4


In [124]:
# Save the cleaned temperature data to CSV
weather_df.to_csv('LaGuardia_Weather_2022_2.0.csv', index=False)

# 05 Merge bike trip and weather data

In [125]:
df2022.dtypes

ride_id                       object
rideable_type               category
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name          category
start_station_id            category
end_station_name            category
end_station_id              category
start_lat                    float32
start_lng                    float32
end_lat                      float32
end_lng                      float32
member_casual               category
month                         object
dtype: object

In [126]:
# Extract only the date part (YYYY-MM-DD) from 'started_at' into a new column

df2022['date'] = pd.to_datetime(df2022['started_at'], format='%Y-%m-%d').dt.date

In [127]:
# Convert 'date' from string to datetime

df2022['date'] = pd.to_datetime(df2022['date'])

In [128]:
df2022.dtypes

ride_id                       object
rideable_type               category
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name          category
start_station_id            category
end_station_name            category
end_station_id              category
start_lat                    float32
start_lng                    float32
end_lat                      float32
end_lng                      float32
member_casual               category
month                         object
date                  datetime64[ns]
dtype: object

In [129]:
# Merge dataframes

df_merged = df2022.merge(weather_df, how = 'left', on = 'date', indicator = True)

In [130]:
df_merged['_merge'].value_counts(dropna = False)

both          29806700
left_only            0
right_only           0
Name: _merge, dtype: int64

In [131]:
# Save the merged df to pickle
df_merged.to_pickle('df_merged_2.0.pkl')

In [132]:
df_merged.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'month', 'date', 'avgTemp', 'temp_max', 'temp_min',
       'total_precipitation', 'wind_speed', '_merge'],
      dtype='object')